# Journey to LLM and prompt engineering

## The Chat Format

In this notebook, you will explore how you can utilize the chat format to have extended conversations with chatbots personalized or specialized for specific tasks or behaviors.

### Setup

In [1]:
import os
import openai
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.getenv('OPENAI_API_KEY')
client = OpenAI()

import os
from helper import load_mistral_api_key
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage

In [2]:
"""
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )
#     print(str(response.choices[0].message))
    return response.choices[0].message["content"]
"""

def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    chat_completion = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return chat_completion.choices[0].message.content

def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
    chat_completion = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature
    )
    return chat_completion.choices[0].message.content

def mistral(user_message, 
            model="mistral-medium-latest",
            is_json=False):
    client = MistralClient(api_key=os.getenv("MISTRAL_API_KEY"))
    messages = [ChatMessage(role="user", content=user_message)]

    if is_json:
        chat_response = client.chat(
            model=model, 
            messages=messages,
            response_format={"type": "json_object"})
    else:
        chat_response = client.chat(
            model=model, 
            messages=messages)
        
    return chat_response.choices[0].message.content

def mistral_from_messages(messages, 
            model="mistral-large-latest",
            temperature=0.0,
            is_json=False):
    client = MistralClient(api_key=os.getenv("MISTRAL_API_KEY"))

    if is_json:
        chat_response = client.chat(
            model=model, 
            messages=messages,
            temperature=temperature,
            response_format={"type": "json_object"})
    else:
        chat_response = client.chat(
            model=model, 
            temperature=temperature,
            messages=messages)
        
    return chat_response.choices[0].message.content

#### "temperature=1" means maximum diverse and creative!  Each time the response would be totally different.

In [10]:
messages =  [  
{'role':'system', 'content':'You are an assistant that speaks like Shakespeare.'},    
{'role':'user', 'content':'tell me a joke'},   
{'role':'assistant', 'content':'Why did the chicken cross the road'},   
{'role':'user', 'content':'I don\'t know'}  ]

#response = get_completion_from_messages(messages, temperature=1)
response = mistral_from_messages(messages, temperature=1)
print(response)

Forsooth, 'tis a query as old as time itself: Why did the chicken traverse the thoroughfare? To seek adventures anon on the other side!


In [4]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},    
{'role':'user', 'content':'Hi, my name is Isa'}  ]

#response = get_completion_from_messages(messages, temperature=1)
response = mistral_from_messages(messages, temperature=1)
print(response)

Hello Isa! It's nice to meet you. How are you doing today? Feel free to chat with me about anything you'd like. 😊


### Didn't mention names.

In [5]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},    
{'role':'user', 'content':'Yes,  can you remind me, What is my name?'}  ]

#response = get_completion_from_messages(messages, temperature=1)
response = mistral_from_messages(messages, temperature=1)
print(response)

Of course! However, you haven't shared your name with me yet. Would you like to tell me your name?


In [7]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},
{'role':'user', 'content':'Hi, my name is Isa'},
{'role':'assistant', 'content': "Hi Isa! It's nice to meet you. \
Is there anything I can help you with today?"},
{'role':'user', 'content':'Yes, you can remind me, What is my name?'}  ]

#response = get_completion_from_messages(messages, temperature=1)
response = mistral_from_messages(messages, temperature=1)
print(response)

Of course! Your name is Isa. It's a pleasure to chat with you, Isa.


## Zero-shot prompting

In [8]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},
{'role':'user', 'content':'Classify the text into positive, neutral or negative:\
Text: That shot selection was awesome.\
Classification:'}  ]

#response = get_completion_from_messages(messages, temperature=1)
response = mistral_from_messages(messages, temperature=1)
print(response)

Based on the text "That shot selection was awesome," the classification would be:

**Positive**

The word "awesome" indicates a positive sentiment.


## Few-shot prompting

In [9]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},
{'role':'user', 'content':'Text: Today the weather is fantastic\
Classification: Pos\
Text: The furniture is small.\
Classification: Neu\
Text: I do not like your attitude\
Classification: Neg\
Text: That shot selection was awful\
Classification:'}  ]

#response = get_completion_from_messages(messages, temperature=1)
response = mistral_from_messages(messages, temperature=1)
print(response)

Neg

(The text expresses a negative sentiment about the shot selection.)


## Chain-of-thought prompting

In [10]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},
{'role':'user', 'content':'The cafeteria had 23 apples. \
If they used 20 to make lunch and bought 6 more, how many apples do they have? \
Let us think step by step.'}  ]

#response = get_completion_from_messages(messages, temperature=1)
response = mistral_from_messages(messages, temperature=1)
print(response)

Sure, let's break it down step by step:

1. The cafeteria started with 23 apples.
2. They used 20 apples to make lunch, so we subtract those from the initial amount:
   23 - 20 = 3 apples remaining.
3. Then, they bought 6 more apples, so we add those to the remaining amount:
   3 + 6 = 9 apples.

So, the cafeteria has 9 apples now.


In [11]:
delimiter = "####"
system_message = f"""
Follow these steps to answer the customer queries.
The customer query will be delimited with four hashtags,\
i.e. {delimiter}. 

Step 1:{delimiter} First decide whether the user is \
asking a question about a specific product or products. \
Product cateogry doesn't count. 

Step 2:{delimiter} If the user is asking about \
specific products, identify whether \
the products are in the following list.
All available products: 
1. Product: TechPro Ultrabook
   Category: Computers and Laptops
   Brand: TechPro
   Model Number: TP-UB100
   Warranty: 1 year
   Rating: 4.5
   Features: 13.3-inch display, 8GB RAM, 256GB SSD, Intel Core i5 processor
   Description: A sleek and lightweight ultrabook for everyday use.
   Price: $799.99

2. Product: BlueWave Gaming Laptop
   Category: Computers and Laptops
   Brand: BlueWave
   Model Number: BW-GL200
   Warranty: 2 years
   Rating: 4.7
   Features: 15.6-inch display, 16GB RAM, 512GB SSD, NVIDIA GeForce RTX 3060
   Description: A high-performance gaming laptop for an immersive experience.
   Price: $1199.99

3. Product: PowerLite Convertible
   Category: Computers and Laptops
   Brand: PowerLite
   Model Number: PL-CV300
   Warranty: 1 year
   Rating: 4.3
   Features: 14-inch touchscreen, 8GB RAM, 256GB SSD, 360-degree hinge
   Description: A versatile convertible laptop with a responsive touchscreen.
   Price: $699.99

4. Product: TechPro Desktop
   Category: Computers and Laptops
   Brand: TechPro
   Model Number: TP-DT500
   Warranty: 1 year
   Rating: 4.4
   Features: Intel Core i7 processor, 16GB RAM, 1TB HDD, NVIDIA GeForce GTX 1660
   Description: A powerful desktop computer for work and play.
   Price: $999.99

5. Product: BlueWave Chromebook
   Category: Computers and Laptops
   Brand: BlueWave
   Model Number: BW-CB100
   Warranty: 1 year
   Rating: 4.1
   Features: 11.6-inch display, 4GB RAM, 32GB eMMC, Chrome OS
   Description: A compact and affordable Chromebook for everyday tasks.
   Price: $249.99

Step 3:{delimiter} If the message contains products \
in the list above, list any assumptions that the \
user is making in their \
message e.g. that Laptop X is bigger than \
Laptop Y, or that Laptop Z has a 2 year warranty.

Step 4:{delimiter}: If the user made any assumptions, \
figure out whether the assumption is true based on your \
product information. 

Step 5:{delimiter}: First, politely correct the \
customer's incorrect assumptions if applicable. \
Only mention or reference products in the list of \
5 available products, as these are the only 5 \
products that the store sells. \
Answer the customer in a friendly tone.

Use the following format:
Step 1:{delimiter} <step 1 reasoning>
Step 2:{delimiter} <step 2 reasoning>
Step 3:{delimiter} <step 3 reasoning>
Step 4:{delimiter} <step 4 reasoning>
Response to user:{delimiter} <response to customer>

Make sure to include {delimiter} to separate every step.
"""


In [14]:
user_message = f"""
by how much is the BlueWave Chromebook more expensive \
than the TechPro Desktop"""

messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message}{delimiter}"},  
] 

#response = get_completion_from_messages(messages)
response = mistral_from_messages(messages, temperature=1)
print(response)

Step 1:#### The user is asking a question about specific products.
Step 2:#### The products mentioned are "BlueWave Chromebook" and "TechPro Desktop," both of which are in the provided list.
Step 3:#### The user assumes that the BlueWave Chromebook is more expensive than the TechPro Desktop.
Step 4:#### The assumption is incorrect. The TechPro Desktop is priced at $999.99, whereas the BlueWave Chromebook is priced at $249.99.
Response to user:#### It seems there might be a misunderstanding. The BlueWave Chromebook is actually priced at $249.99, while the TechPro Desktop is priced at $999.99. So, the TechPro Desktop is more expensive than the BlueWave Chromebook.


In [15]:
user_message = f"""
do you sell tvs"""
messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message}{delimiter}"},  
] 

#response = get_completion_from_messages(messages)
response = mistral_from_messages(messages, temperature=1)
print(response)

Step 1:#### The user is not asking about a specific product.

Response to user:#### I'm sorry for any confusion, but we currently only offer computers and laptops for sale. Is there anything specific you're looking for in that category? I'd be happy to help!


## OrderBot
We can automate the collection of user prompts and assistant responses to build a  OrderBot. The OrderBot will take orders at a pizza restaurant. 

In [13]:
def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_from_messages(context) 
    #response = mistral_from_messages(context) 
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, style={'background-color': '#F6F6F6'})))
 
    return pn.Column(*panels)


In [14]:
import panel as pn  # GUI
pn.extension()

panels = [] # collect display 

context = [ {'role':'system', 'content':"""
You are OrderBot, an automated service to collect orders for a pizza restaurant. \
You first greet the customer, then collects the order, \
and then asks if it's a pickup or delivery. \
You wait to collect the entire order, then summarize it and check for a final \
time if the customer wants to add anything else. \
If it's a delivery, you ask for an address. \
Finally you collect the payment.\
Make sure to clarify all options, extras and sizes to uniquely \
identify the item from the menu.\
You respond in a short, very conversational friendly style. \
The menu includes \
pepperoni pizza  12.95, 10.00, 7.00 \
cheese pizza   10.95, 9.25, 6.50 \
eggplant pizza   11.95, 9.75, 6.75 \
fries 4.50, 3.50 \
greek salad 7.25 \
Toppings: \
extra cheese 2.00, \
mushrooms 1.50 \
sausage 3.00 \
canadian bacon 3.50 \
AI sauce 1.50 \
peppers 1.00 \
Drinks: \
coke 3.00, 2.00, 1.00 \
sprite 3.00, 2.00, 1.00 \
bottled water 5.00 \
"""} ]  # accumulate messages


inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard


/var/folders/7t/drc_5mxn3c96k2_3zc1n57yh0000gn/T/ipykernel_92497/3526675236.py:11: PanelDeprecationWarning: 'style' is deprecated and will be removed in version 1.4, use 'styles' instead.
  pn.Row('Assistant:', pn.pane.Markdown(response, width=600, style={'background-color': '#F6F6F6'})))


Column
    [0] TextInput(placeholder='Enter text here…')
    [1] Row
        [0] Button(name='Chat!')
    [2] ParamFunction(function, _pane=Column, defer_load=False, height=300, loading_indicator=True)

In [16]:
messages =  context.copy()
#print(messages)

messages.append(
{'role':'system', 'content':'create a json summary of the previous food order. Itemize the price for each item\
 The fields should be 1) pizza, include size 2) list of toppings 3) list of drinks, include size   4) list of sides include size  5)total price '},    
)
 #The fields should be 1) pizza, price 2) list of toppings 3) list of drinks, include size include price  4) list of sides include size include price, 5)total price '},    

response = get_completion_from_messages(messages, temperature=0)
#response = mistral_from_messages(messages, temperature=1, is_json=True)
print(response)

{
  "pizza": {
    "size": "medium cheese pizza",
    "price": 9.25
  },
  "toppings": [
    {
      "name": "mushrooms",
      "price": 1.50
    }
  ],
  "drinks": [
    {
      "name": "small sprite",
      "price": 1.00
    }
  ],
  "sides": [],
  "total price": 11.75
}


# No longer works !!!
- Something wrong with panel package.

## Mandarin Idiom Explainer

The notebook contains snippets from both CS50 and Andrew Ng Prompting tutorials:

- https://www.youtube.com/watch?v=vw-KWfKwvTQ&t=286s
- https://github.com/yu-to-chen/generative-ai/blob/main/langchain/7-chatbot_Chinese.ipynb

It is a primitive chatbot capable of responding to every situation with a Chinese 成語 and provide a word of encouragement for learning languages.

In [11]:
import panel as pn  # GUI
pn.extension()

panels = [] # collect display 

context = [ {'role':'system', 'content':"""
You are a ChineseIdiomBot, a kind, helpful Chinese teacher.
You first respond to every situation by explaining the 成語 that fits it.
Then provide a note of encouragement about learning languages.
"""} ]  # accumulate messages


In [12]:
"""
inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    #pn.Row('Assistant:', pn.pane.Markdown(response, width=600, style={'background-color': '#F6F6F6'})),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard
"""

'\ninp = pn.widgets.TextInput(value="Hi", placeholder=\'Enter text here…\')\nbutton_conversation = pn.widgets.Button(name="Chat!")\n\ninteractive_conversation = pn.bind(collect_messages, button_conversation)\n\ndashboard = pn.Column(\n    inp,\n    pn.Row(button_conversation),\n    #pn.Row(\'Assistant:\', pn.pane.Markdown(response, width=600, style={\'background-color\': \'#F6F6F6\'})),\n    pn.panel(interactive_conversation, loading_indicator=True, height=300),\n)\n\ndashboard\n'